In [2]:
%autosave 120


Autosaving every 120 seconds


In [3]:
import pandas as pd
import xlrd
import numpy as np
import openpyxl as opx
import glob
import sys
import datetime
import xlwings as xl
import copy
from openpyxl.utils import range_boundaries
from fuzzywuzzy import fuzz
import pyodbc
import math as math
path = 'C:\\trial'
ticker_list =[] 
folder = glob.glob(path + '\\*')
company = [1]
import ast
c = 0
import configparser
con=configparser.ConfigParser()
con.read("C:\\Users\\admin\\thapovan\\Thapovan2\\Config.cfg")

C:\Users\admin\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


['C:\\Users\\admin\\thapovan\\Thapovan2\\Config.cfg']

In [4]:
import pandas as pd
import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import PorterStemmer
stemmer = PorterStemmer()
import numpy as np
np.random.seed(2018)
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
lem = WordNetLemmatizer()
stem = PorterStemmer()
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
stop_words = set(stopwords.words("english"))
new_words = ["using", "show", "result", "large", "also", "iv", "one", "two", "new", "previously", "shown"]
stop_words = stop_words.union(new_words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
os.chdir("C:\\Users\\admin\\thapovan\\Thapovan2\\Corpus")
corpus_list = glob.glob("C:\\Users\\admin\\thapovan\\Thapovan2\\Corpus\*csv")

In [6]:
data1 = pd.read_csv("values1.csv", error_bad_lines=False)
data = pd.read_csv("Thapovan_data.csv", error_bad_lines=False)

In [7]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [8]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [9]:
def return_match_list(db,trial):
    p_list = []
    for i in range(0,len(trial)):
        metric = trial['trial'][i]
        text = re.sub('[^a-zA-Z]', ' ', metric)
    
        #Convert to lowercase
        text = text.lower()
    
        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
    
        ##Convert to list from string
        text = text.split()
    
        ##Stemming
        ps=PorterStemmer()
        #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
        text = " ".join(text)
        tex_check = text.split(" ")
        list=[]
        for key in data1["Keyword"]:
            count = 0
            for j in tex_check:
                if(j in key):
                    count = count+1
            list.append((count/len(tex_check))*100)  
        l = []
        count1 = 0 
        maxi = max(list)
        for val in list:
            if(val==maxi):
                l.append(count1)
            count1 = count1+1 
        l2=[]
        for j in l:
            l2.append(data['Term'][j])
        p_list.append(l2)      
    trial['possible'] = p_list
    t_list = []
    for i in range(0,len(db)):
        metric = db['metric_name'][i]
        text = re.sub('[^a-zA-Z]', ' ', metric)
    
        #Convert to lowercase
        text = text.lower()
    
        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
    
        ##Convert to list from string
        text = text.split()
    
        ##Stemming
        ps=PorterStemmer()
        #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
        text = " ".join(text)
        tex_check = text.split(" ")
        list=[]
        for key in data1["Keyword"]:
            count = 0
            for j in tex_check:
                if(j in key):
                    count = count+1
            list.append((count/len(tex_check))*100)  
        l = []
        count1 = 0 
        maxi = max(list)
        for val in list:
            if(val==maxi):
                l.append(count1)
            count1 = count1+1 
        l2=[]
        for j in l:
            l2.append(data['Term'][j])
        t_list.append(l2)    
    db['possible'] = t_list    
    final_list = []
    fill = []
    for i in range(0,len(trial)):
        metric = trial['possible'][i]
        final = []
        for j in range(0,len(db)):
            count = 0
            for k in db['possible'][j]:
                if(k in metric):
                    count = count+1
            final.append((count/len(db['possible'][j]))*100)
            l3 =[]
            for k in range(0,len(final)):
                if(final[k]>0):
                    l3.append(db['metric_name'][k])
        if(len(l3)==len(final)):
            final_list.append(fill)
        else:    
            final_list.append(l3) 
    return(final_list)  

In [10]:
def ticker_dictionary_build(tick):
    #db_metrics = input("enter the name of the excel sheet with the database metrics for this particular ticker")
    #analyst_metrics = input("enter the name of the excel sheet with the analyst metrics for this particular ticker")
    print("Choose the name of the csv with the metrics from the database from the gven list. Enter the corresponding index number")
    for i in range(0,len(corpus_list)):
        print(str(i) +')' + corpus_list[i])
    db_metrics=corpus_list[int(input("Enter your choice:"))]
    print("Choose the name of the csv with the metrics from the analysts from the gven list. Enter the corresponding index number")
    for i in range(0,len(corpus_list)):
        print(str(i) +')' + corpus_list[i])
    analyst_metrics=corpus_list[int(input("Enter your choice:"))]
    db = pd.read_csv(db_metrics, error_bad_lines=False)
    trial = pd.read_csv(analyst_metrics, error_bad_lines=False)
    final_matches = return_match_list(db,trial)
    trial['matches'] = final_matches
    csv_name = tick + "_matches.csv"
    trial.to_csv(csv_name)
    return()

In [55]:
#connection to database:
server = ast.literal_eval(con.get("Setting","server"))
database = ast.literal_eval(con.get("Setting","database"))
username = ast.literal_eval(con.get("Setting","username"))
password = ast.literal_eval(con.get("Setting","password"))
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [12]:

data = []
#function which get information of file using its path and return 5 elements(year, month,date,ticker code,analyst name)
def formating(filen):
    data = filen.replace(path, '').split('\\')
    del data[0]
    del data[3]
    if '-' in data[-1]:
        temp = (data[-1].split('.')[0]).split('-')
    elif '_' in data[-1]:
        temp = (data[-1].split('.')[0]).split('_')
    else:
        temp = (data[-1].split('.')[0]).split(' ')
    del data[-1]
    data.append(temp[0])
    data.append(temp[1])
    print(data)
    return(data)

In [13]:

#defined variable which help us to recognize year and quater of given column

year_pattern_int=[]
year_pattern_float=[]
year_pattern_2di=[]
quater_pattern=ast.literal_eval(con.get("Setting","quater_pattern"))
bscf_names=ast.literal_eval(con.get("Setting","bscf_names"))
#bscf_names=["balancesheet","cashflows","bs&cf","cf&bs","bsheet","balancesheet&cashflow","cashflow&balance sheet","cash&balance","adjustedbs","annualbs&cf","annualcf&bs"]

check_list = ast.literal_eval(con.get("Setting","check_list"))
for i in range(2000,2031):
    year_pattern_int.append(i)
    year_pattern_float.append(float(i))
    if i-2000<10:
        year_pattern_2di.append('0'+str(i-2000))
    else:
        year_pattern_2di.append(str(i-2000))


In [59]:
# type of main function

def function1(excel_file,path_list):
    #read excel file using openpyxl
    wb=opx.load_workbook(excel_file,data_only=True)
    sheet_tobechecked=wb.sheetnames #getting all the sheet name
    nonhidden=[]
    startrow={}
    print(sheet_tobechecked)
    #this part helps to ignore hidden sheets and to take related to BS , Cf and IS
    names=[]
    for i in range(0, len(sheet_tobechecked)):
        curr_sheet = wb[sheet_tobechecked[i]]
        if curr_sheet.sheet_state != 'hidden':
            nonhidden.append(sheet_tobechecked[i])
            if sheet_tobechecked[i].lower()=='bs' or sheet_tobechecked[i].lower()=='cf' or sheet_tobechecked[i].lower()=='model':
                names.append(sheet_tobechecked[i])
            else:
                count_nobscf=1
                for nam in bscf_names:
                    if fuzz.partial_token_set_ratio(sheet_tobechecked[i].lower(),nam)>=80:
                        names.append(sheet_tobechecked[i])
    
    sheet_tobechecked = names # all the sheet which are important to us
    
    #if no sheet found then return giving error as sheet_name_fault
    if len(sheet_tobechecked)<=0:
        return (2) # that means there is problem
        '''print(nonhidden)
        while True:
            sheee=input("Enter the sheet name U want to process from above or for exit input'e' ")
            if sheee!='e':
                sheet_tobechecked.append(sheee)
            else:
                break'''
    
    sheet_tobechecked=list(set(sheet_tobechecked))# to remove repeated sheet names
    print(sheet_tobechecked)
    print("sheet to be checked")
        
    #defining variable
    year_pre,quater_pre,q4_check_pre='','',0
    
    #unmerging all cells of every sheet of a file
    for every_sheet in sheet_tobechecked:
        
        startrow_no=startingrow(wb,every_sheet)-1
        if startrow_no<0:
            startrow[every_sheet]=0
        else:
            startrow[every_sheet]=startrow_no
        
        curr_sheet=wb[every_sheet]
        merge_list=copy.deepcopy(curr_sheet.merged_cells.ranges)
        for group in merge_list :
            split_arr = group.coord.split(':')
            min_col, min_row, max_col, max_row = range_boundaries(group.coord)
            top_left_cell_value = curr_sheet[split_arr[0]].value
            curr_sheet.unmerge_cells(group.coord)
            for row in curr_sheet.iter_rows(min_col=min_col, min_row=min_row, max_col=max_col, max_row=max_row):
                for cell in row:
                    cell.value = top_left_cell_value
    wb.save(excel_file) #the modified file is been overwritten (have to look over it afterward by changing path)
    print("excel")
    display(excel_file)
    for sheetno in range(0,len(sheet_tobechecked)):
        print(sheet_tobechecked[sheetno])
        address_dataframe=pd.DataFrame(columns=['q1','q2','q3','q4','tl','Ex']) # defining lookup table
        excel_data = pd.read_excel(excel_file,sheet_name=sheet_tobechecked[sheetno],header=None,skiprows=[i for i in range(0,startrow[sheet_tobechecked[sheetno]])])
        pd.options.display.max_columns = None
        pd.options.display.max_rows=None
        print("This is excel data")
        display(excel_data)
        
        #formating the dataframe by removing column and row having less than 10 element
        excel_data=excel_data.replace(0,np.nan)
        excel_data=excel_data.dropna(axis=0,thresh=10)
        excel_data=excel_data.dropna(axis=1,thresh=10)
        excel_data=excel_data.reset_index(drop=True)
        if(len(excel_data.columns)==0):
            continue # this line was added to handle exceptions in the code
        excel_data.columns = range(excel_data.shape[1])
        pd.options.display.max_columns = None
        pd.options.display.max_rows=None
        display(excel_data)
        startrowno=0
        
        #getting defination of each column
        print(len(excel_data.columns))
        if len(excel_data.columns)>10: #changed from 10 to 3
            temp_dataframe=excel_data[:10] # the first 10 element of a column will tell us about it 
            for j in temp_dataframe:
                list_ele=list(temp_dataframe[j])
                knowledge=[]
                colno=j
                for i in list_ele:
                    #overlook element if its nan,none, or weeks or days written in it
                    if str(i)=='nan' or str(i)=='None' or type(i)==datetime.datetime or "week" in str(i) or "day" in str(i):
                        a=1
                    #take only element which are keywords to us (like FY2017,3Q17 etc.) or else break which means any value have been occured
                    elif (type(i)==float and i not in year_pattern_float) or (type(i)==int and i not in year_pattern_int):
                        startrowno=list_ele.index(i)
                        break
                    else:
                        knowledge.append(i)
                
                # proces further if the list taken for defining the row is not empty 
                if len(knowledge)>0:
                    
                    year_got,quater_got,q4_check_got=about_q_year(knowledge)
                        
                    #here is a small assumtion (if we got Q4 and all the elementused to define column is same we will take the curent row as Total of particuar year) )
                    if q4_check_got==1 and year_got==year_pre and quater_got==quater_pre and q4_check_got==q4_check_pre:
                        year_final=year_got
                        quater_final='TL'
                        q4_check_final=1
                    else:
                        # or else everything will be consider as it was given has been found about it
                        year_pre=year_got
                        quater_pre=quater_got
                        q4_check_pre=q4_check_got
                        year_final=year_got
                        quater_final=quater_got
                        q4_check_final=q4_check_got                       
                    
                    #if only year is written and nothing about the quater we take that as Total for particular year
                    if year_got!='' and (quater_got=='' or quater_got=='TL' or quater_pre=='TL' or quater_pre==''):
                        quater_final='TL'
                    
                    #printing and making dataframe(lookup table) of information of each columns
                    if year_final!='' and quater_final!='':
                        got_cor=1 #means we got something
                        #print(knowledge,"===",end="")
                        #print("belongs to "+"20"+year_final," "+quater_final)
                        if '1' in quater_final:
                            qval='q1'
                        elif '2' in quater_final:
                            qval='q2'
                        elif '3' in quater_final:
                            qval='q3'
                        elif '4' in quater_final:
                            qval='q4'
                        else:
                            qval='tl'
                        try:
                            address_dataframe.loc["20"+year_final][qval]=[startrowno,colno]
                        except Exception:
                            address_dataframe.loc["20"+year_final]='Nan'
                            address_dataframe.loc["20"+year_final][qval]=[startrowno,colno]                           
                 
        display(address_dataframe)
        #db_connect()
        #extra fro shoeing rows which doesnot match
        not_present=pd.DataFrame(columns=excel_data.columns)
        coll=0
        
        #this is to make a list of year and its respective column number from lookup take which has to consider
        row_name_year=list(address_dataframe.index)
        row_name_year.sort()
        if int(path_list[0])<int(row_name_year[-1]):
            cur_yr=int(path_list[0])
            count_yr=0
            count = 0
            metric_col=[]
            par_year=[]
            metric_row=20
            year_tosearch=cur_yr
            
            lowest_year=min_year("'"+path_list[3]+"'")

            while (count_yr<6 and year_tosearch>=lowest_year):
                year_tosearch=cur_yr-(count_yr+1)
                val=address_dataframe['q1'][str(year_tosearch)]
                if val!='Nan':
                    if val[0]<metric_row:
                        metric_row=val[0]
                    metric_col.append(val[1])
                    par_year.append(year_tosearch)
                    count_yr+=1
            print(par_year)
            print(metric_col)
            print(metric_row)
                

            all_metric=set()
            def_metrics = []
            matched_metric = []
            mid_matched =[]
            maybe_metrics = []
            no_metrics = []
            if(type(excel_data.iloc[metric_row][0])==str):
                c = 0
            else:
                c = 1
            metric_list =[]    
            list_subs_found =[]
            for rowss in range(metric_row,excel_data.shape[0]):
                metric_found=[]
                s_var = ['Macau','Venetian']
                list_metric_found=[]
                com_met=set()
                a_metric=str(excel_data.iloc[rowss][c])
                flag_sub = 0
                for i in s_var:
                    if i in a_metric:
                        s = i
                        flag_sub = 1
                if flag_sub==0:
                    if rowss>metric_row+5:
                        for i in range(1,5):
                            for j in s_var:
                                if j in str(excel_data.iloc[rowss-i][c]):
                                    s=j
                                    flag_sub =1
                            if flag_sub==1:    
                                    break
                if flag_sub==0: 
                    s = "Unknown"
                for met_num in range(0,6):
                    #getting the metric name wrt the year,quater,ticker and value send
                    metric,subs_list=get_metric_name(par_year[met_num],path_list[3],excel_data.iloc[rowss][metric_col[met_num]],a_metric)
                    #remove this if it fucks up
                    for i in range(0,len(metric)):
                        metric[i] = metric[i] +"["+str(subs_list[i])+"]"
                    metric = set(metric)
                    subs_list = set(subs_list)
                    com_met.update(metric)
                    metric_list.extend(metric)
                    list_metric_found.extend(metric)
                    list_subs_found.extend(subs_list)
                if com_met:
                    print(com_met)
                    print(list_metric_found)
                for un_met in com_met:
                    #if min 60% of them says that yes the values are of  particlar metric we take it 
                    if list_metric_found.count(un_met)/6 >= 0.6:
                        a = un_met +"--->"+a_metric+ "--->"+ str(rowss) +"--->"+ s
                        metric_found.append(un_met)
                        matched_metric.append(un_met)
                        def_metrics.append(a)
                    elif list_metric_found.count(un_met)/6 >= 0.4 and list_metric_found.count(un_met)/6 <0.6:
                        if(un_met not in matched_metric): 
                            mid_matched.append(un_met)
                            a = un_met +"--->"+a_metric+ "--->"+ str(rowss) +"--->"+ s
                            maybe_metrics.append(a)
                    elif list_metric_found.count(un_met)/6 < 0.4:
                        if(un_met not in matched_metric and un_met not in mid_matched):
                            no_metrics.append(un_met)
                if metric_found:
                    print(rowss,metric_found)
                    all_metric.update(metric_found)
                else:
                    not_present.loc[coll]=list(excel_data.loc[rowss])
                    coll+=1
                
            print(all_metric,len(all_metric))#printing all mertric found in database 
            
            metric_db=get_metriclist_from_db(path_list[0],path_list[3])
            print("metric not in actuals---")
            print(list(set(metric_db)-set(all_metric)))# all metric we couldn't find wrt to a ticker
            print("metric not with respect to analyst")
            print(list(set(get_metriclist_wrt_analyst(path_list[3],path_list[4]))-set(all_metric)))#metric which we couldn't finf wrt to analyst
            # the percentge accuracy will be calculate wby no of metric_found by us divided by total number of metric wrt analyst 
            print()
            print()
            print("ACCURACY==")
            all_metric = list(all_metric)
            print("Number of total element in Actual DB=", len(metric_db))
            print("Number of metric found by us=",len(all_metric))
            for i in range(0,len(all_metric)):
                all_metric[i] = all_metric[i].replace(all_metric[i][all_metric[i].find("[")+1:all_metric[i].find("]")],"").strip("][")
            print("accuracy wrt to Actual=",len(list(set(metric_db)&set(all_metric)))*100/len(metric_db))
            print()
            print("Number of total element in Estimated DB wrt Analyst=", len(get_metriclist_wrt_analyst(path_list[3],path_list[4])))
            z=set(get_metriclist_wrt_analyst(path_list[3],path_list[4]))
            print("Number of metric found by us=",len(z&set(all_metric)))
            print("Accuracy with respect to estimates=",len(z&set(all_metric))*100/len(z))
            print("list of definate metrics", list(set(def_metrics)))
            print()
            print("list of semi-definate metrics", list(set(maybe_metrics)))
            print()
            print("list of Unmatched metrics", list(set(no_metrics)))
            m_metrics = list(set(maybe_metrics))
            d_metrics = list(set(def_metrics))
            print()
            print("The partially matched metrics can be mapped as so")
            print()
            for metric in m_metrics:
                poss = []
                poss = metric.split("--->")
                col=[c]
                for i in range(0,6):
                    col.append(metric_col[i])
                temp_metric = pd.DataFrame()
                temp_metric=temp_metric.append(excel_data.loc[[int(poss[2])],col[0:7]])
                temp_metric.rename(columns={col[0]:'Name',col[1]:str(par_year[0])+'Q1',col[2]:str(par_year[1])+'Q1',col[3]:str(par_year[2])+'Q1',col[4]:str(par_year[3])+'Q1',col[5]:str(par_year[4])+'Q1',col[6]:str(par_year[5])+'Q1'},inplace=True)
                yr = str(par_year[0:6]).strip('][').replace(" ,","")
                display(temp_metric)
                sub = int(poss[0][poss[0].find("[")+1:poss[0].find("]")])
                poss0 = poss[0].replace(poss[0][poss[0].find("[")+1:poss[0].find("]")],"").strip("][")
                select_stmt1 = F"select value from dbo.cc_actual_metrics_consolidated where " \
                    F"metric_name='{poss0}' and subsidiary_id ={sub} and quarter = 'Q1' and ticker_code ='{path_list[3]}' and financial_year in (select convert(int,id) from [adm_fnSplitter]('{yr}')) order by financial_year"  
                cursor.execute(select_stmt1)
                row = cursor.fetchone()
                m_l =[]
                while row:
                    m_l.append(str(row.value))
                    row= cursor.fetchone()
                m_l.reverse()
                print(m_l)    
                print(metric)
                ip = input("enter [Y]es if you think this is a valid match and [N]o if you think it can be bypassed")
                if(ip is "Y"):
                    def_metrics.append(metric)
                else:
                    continue
            #matched_df = pd.DataFrame()
            print("the def matched metrics are")
            for m in d_metrics:
                poss = []
                poss = m.split("--->")
                col=[c]
                an_metric = pd.DataFrame(columns = ['Name',str(par_year[0])+'Q1',str(par_year[1])+'Q1',str(par_year[2])+'Q1',str(par_year[3])+'Q1',str(par_year[4])+'Q1',str(par_year[5])+'Q1'])
                temp_metric = pd.DataFrame()
                columns = ['Name',str(par_year[0])+'Q1',str(par_year[1])+'Q1',str(par_year[2])+'Q1',str(par_year[3])+'Q1',str(par_year[4])+'Q1',str(par_year[5])+'Q1']
                for i in range(0,6):
                    col.append(metric_col[i])
                temp_metric=temp_metric.append(excel_data.loc[[int(poss[2])],col[0:7]])
                temp_metric.rename(columns={col[0]:'Name',col[1]:str(par_year[0])+'Q1',col[2]:str(par_year[1])+'Q1',col[3]:str(par_year[2])+'Q1',col[4]:str(par_year[3])+'Q1',col[5]:str(par_year[4])+'Q1',col[6]:str(par_year[5])+'Q1'},inplace=True)
                #display(temp_metric)  
                inx = metric_list.index(poss[0])
                sub = int(poss[0][poss[0].find("[")+1:poss[0].find("]")])
                poss0 = poss[0].replace(poss[0][poss[0].find("[")+1:poss[0].find("]")],"").strip("][")
                print(sub)
                if sub is not None and sub is not "None":    
                    select_stmt1 = F"select value from dbo.cc_actual_metrics_consolidated where " \
                        F"metric_name = '{poss0}' and subsidiary_id = {sub} and ticker_code = '{path_list[3]}' and quarter = 'Q1' and financial_year in (select convert(int,id) from [adm_fnSplitter]('2012,2013,2014,2015,2016,2017')) order by financial_year"
                else:
                    select_stmt1 = F"select value from dbo.cc_actual_metrics_consolidated where " \
                        F"metric_name = '{poss0}' and ticker_code = '{path_list[3]}' and quarter = 'Q1' and financial_year in (select convert(int,id) from [adm_fnSplitter]('2012,2013,2014,2015,2016,2017')) order by financial_year"
                cursor.execute(select_stmt1)
                row = cursor.fetchone()
                m_l =[]
                m_l1 = [poss0]
                while row:
                    m_l.append(str(row.value))
                    row= cursor.fetchone()
                m_l.reverse()
                m_l1.extend(m_l)
                temp_metric = temp_metric.append(pd.Series(m_l1, index=['Name',str(par_year[0])+'Q1',str(par_year[1])+'Q1',str(par_year[2])+'Q1',str(par_year[3])+'Q1',str(par_year[4])+'Q1',str(par_year[5])+'Q1']), ignore_index=True) 
                display(temp_metric)
                print(m_l)
                print(m)
                #matched_df = matched_df.append(excel_data.loc[[int(poss[2])],col[0:7]], ignore_index = True)
            print()
            #print("The definately matched metrics are")
            print()
            #display(matched_df)
            #print("The matches made are")
            #for m in def_metrics:
            #   print(m)
            #display(not_present)
    
    if got_cor:
        return 0
    else:
        return 1
   
    '''qw=input()
            if qw=='q':
                sys.exit()'''


In [60]:

#get the minimum year present in dtabase for given ticker
def min_year(ticker):
    select_stmt1 = F" select distinct financial_year from dbo.cc_actual_metrics_consolidated where ticker_code={ticker} order by financial_year"
    cursor.execute(select_stmt1)
    row = cursor.fetchone()
    res_array = []
    while row:
        res_array.append(row[0])
        row = cursor.fetchone()
    print(res_array[0])
    return(res_array[0])


In [61]:
def get_metric_name (year,ticker,value,a_metric,quarter='Q1'):
    flag = 0
    flag_not_found = 1
    possible_list = ""
    try:
        index = ticker_dictionary[ticker_dictionary['trial']==a_metric].index.item()
        possible_list = ticker_dictionary['matches'][index]
        param = possible_list.strip("][").replace("'","").replace(", ",",").replace(" ,",",")
        flag = 1
    except:
        pass
    #Also write a conditional if statement for the case of an empty list 
    ticker_code = "'"+ticker+"'"
    year = year
    quarter = "'"+quarter+"'"
    if type(value)==int or (type(value)==float and str(value)!='nan'):
        if abs(value)>=1:
            #when value is greater than 1
            valueo= "'"+str(value)+"'" #orginal value
            value1 = "'"+str(int(value))+".%"+"'" # integered value
            value2 = "'"+str(round(value, 2))+"%"+"'" #value when rounded upto 2
            value3 = "'"+str(-1*int(value))+".%"+"'" #integered with negation
            value4 = "'"+str(-1*round(value,2))+"%"+"'" # round upto 2 with negation
            print(valueo,value1,value2,value3,value4)
            if(len(possible_list)>2 and flag==1):
                select_stmt1 = F"select distinct metric_name, subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"metric_name in (select id from [adm_fnSplitter]('{param}')) and " \
                    F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                    F"(value like {valueo} or value like {value1} or value like {value2} or value like {value3} or value like {value4})"   
            else:
                select_stmt1 = F"select distinct metric_name, subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                    F"(value like {valueo} or value like {value1} or value like {value2} or value like {value3} or value like {value4})"
            subs_list = []    
            metric_list=[]
            cursor.execute(select_stmt1)
            row = cursor.fetchone()
            while row:
                flag_not_found = 0
                metric_list.append(row.metric_name)
                subs_list.append(row.subsidiary_id)
                row = cursor.fetchone()
        else:
            #if value less than 1
            valueo= "'"+str(value*100)+"'" #orginal with 100 multiplied
            value1 = "'"+str(int(value*100))+".%"+"'" # multiply by 100 and then integered
            value3 = "'"+str(-1*int(value*100))+".%"+"'" # multiply by 100 and integer and negate it
            print("with 100",valueo,value1,value3)
            if(len(possible_list)>2 and flag==1):
                select_stmt1 = F"select distinct metric_name, subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"metric_name in (select id from [adm_fnSplitter]('{param}')) and " \
                    F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                    F"(value like {valueo} or value like {value1} or value like {value3})"
            else:
                select_stmt1 = F"select distinct metric_name, subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                    F"(value like {valueo} or value like {value1} or value like {value3})"
            subs_list = []    
            metric_list=[]
            cursor.execute(select_stmt1)
            row = cursor.fetchone()
            flag_got_with_100=0
            while row:
                metric_list.append(row.metric_name)
                subs_list.append(row.subsidiary_id)
                row = cursor.fetchone()
                flag_got_with_100=1

            #if not flag_got_with_100:
            valueo = "'"+str(value)+"'" #orginal value
            value1 = "'"+str(round(value,2))+"%"+"'" # round upto 2
            value3 = "'"+str(-1*round(value,2))+"%"+"'" # round upto 2  and negate it
            print("without 100",valueo,value1,value3)
            if(len(possible_list)>2 and flag==1):
                select_stmt1 = F"select distinct metric_name, subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"metric_name in (select id from [adm_fnSplitter]('{param}')) and " \
                    F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                    F"(value like {valueo} or value like {value1} or value like {value3})"
            else:
                select_stmt1 = F"select distinct metric_name, subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                    F"(value like {valueo} or value like {value1} or value like {value3})"
            cursor.execute(select_stmt1)
            row = cursor.fetchone()
            while row:
                #to_confirm
                metric_list.append(row.metric_name)
                subs_list.append(row.subsidiary_id)
                row = cursor.fetchone()
                flag_not_found = 0
        if(flag_not_found == 1 and flag == 1):
            if abs(value)>=1:
                #when value is greater than 1
                valueo= "'"+str(value)+"'" #orginal value
                value1 = "'"+str(int(value))+".%"+"'" # integered value
                value2 = "'"+str(round(value, 2))+"%"+"'" #value when rounded upto 2
                value3 = "'"+str(-1*int(value))+".%"+"'" #integered with negation
                value4 = "'"+str(-1*round(value,2))+"%"+"'" # round upto 2 with negation
                select_stmt1 = F"select distinct metric_name, subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"metric_name not in (select id from [adm_fnSplitter]('{param}')) and " \
                    F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                    F"(value like {valueo} or value like {value1} or value like {value2} or value like {value3} or value like {value4})"
                metric_list=[]
                subs_list = []
                cursor.execute(select_stmt1)
                row = cursor.fetchone()
                while row:
                    metric_list.append(row.metric_name)
                    subs_list.append(row.subsidiary_id)
                    row = cursor.fetchone()
            else:
                #if value less than 1
                valueo= "'"+str(value*100)+"'" #orginal with 100 multiplied
                value1 = "'"+str(int(value*100))+".%"+"'" # multiply by 100 and then integered
                value3 = "'"+str(-1*int(value*100))+".%"+"'" # multiply by 100 and integer and negate it  
                select_stmt1 = F"select distinct metric_name, subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"metric_name not in (select id from [adm_fnSplitter]('{param}')) and " \
                    F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                    F"(value like {valueo} or value like {value1} or value like {value3})"
                metric_list=[]
                subs_list = []
                cursor.execute(select_stmt1)
                row = cursor.fetchone()
                flag_got_with_100=0
                while row:
                    metric_list.append(row.metric_name)
                    subs_list.append(row.subsidiary_id)
                    row = cursor.fetchone()
                    flag_got_with_100=1
                valueo = "'"+str(value)+"'" #orginal value
                value1 = "'"+str(round(value,2))+"%"+"'" # round upto 2
                value3 = "'"+str(-1*round(value,2))+"%"+"'" # round upto 2  and negate it
                select_stmt1 = F"select distinct metric_name, subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"metric_name not in (select id from [adm_fnSplitter]('{param}')) and " \
                    F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                    F"(value like {valueo} or value like {value1} or value like {value3})"   
                cursor.execute(select_stmt1)
                row = cursor.fetchone()
                while row:
                    #to_confirm
                    metric_list.append(row.metric_name)
                    subs_list.append(row.subsidiary_id)
                    row = cursor.fetchone()
        return(metric_list,subs_list)
    else:
        return([],[])


In [62]:
#function which tells about the quater and year the particular column belongs to
def about_q_year(checkar):
    
    checkar_duplicate=np.copy(checkar)
    quater=''
    year=''
    year_flag=0
    quater_flag=0
    q4_check=0
    checkar=[]
    datetime_flag=1
    for i_che in checkar_duplicate:
        checkar.append(str(i_che))
    
    for i_che in range(0,len(checkar)):
        
        if not year_flag:
            #checking if year is presenr in (2017,2016 format)
            for j_y in year_pattern_int:
                if str(j_y) in checkar[i_che]:
                    year=str(j_y)[2:]
                    checkar[i_che]=checkar[i_che].replace(str(j_y),'')
                    checkar[i_che]=checkar[i_che].replace(year,'')
                    year_flag=1
                    break
        if not year_flag:
            #checking if the year is present in 2 digited format like 17,16)
            for j_y in year_pattern_2di:
                if str(j_y) in checkar[i_che]:
                    year=str(j_y)
                    checkar[i_che]=checkar[i_che].replace(year,'')
                    year_flag=1
                    break

        if not quater_flag:
            # getting info about the quater
            for i_p in quater_pattern:
                checkar[i_che]=checkar[i_che].replace(year,'')
                checkar[i_che]=checkar[i_che].replace("20"+year,'')
                if i_p in checkar[i_che].lower():
                    quater=i_p
                    quater_flag=1
                    break       
        
        if year_flag and quater_flag:
            break
    #returning the year and quater and if its 4th quatered or not
    return(year,quater,q4_check)


In [63]:
# funtion to avoid te rough work present in top and retutn the row no from which its important
def startingrow(wb,sheet_name):
    curr_sheet = wb[sheet_name]
    flag = 0
    row = 0
    for col in curr_sheet.iter_cols(max_col=20,max_row=15):
        for cell in col:
            if cell.value is not None:
                for data in check_list:
                    if fuzz.partial_ratio(str(data),str(cell.value)) == 100:
                        row = cell.row
                        flag =1
                        break
                if flag == 1:
                    break
        if flag == 1:
            break

    print(row)
    return(row)

In [64]:
#function which return all the metric anme with repect to ticker in actuals
def get_metriclist_from_db(year, ticker):

    ticker_code = "'"+ticker+"'"
    year = year
    select_stmt1 = F"select distinct metric_name from dbo.cc_actual_metrics_consolidated where ticker_code = {ticker_code} " \
        F" and financial_year = {year}"

    cursor.execute(select_stmt1)
    row = cursor.fetchone()
    metric_list_comp = []
    while row:
        metric_list_comp.append(row[0])
        row = cursor.fetchone()

    return (metric_list_comp)

In [65]:
#function which return aall the metric name wrt to analyst from estimated database
def get_metriclist_wrt_analyst(ticker,analyst):
    ticker_code = "'"+ticker+"'"
    analyst_name="'%"+analyst+"%'"
    select_stmt1 = F" select distinct metric_name from [dbo].[cc_metrics_consolidated] where ticker_code = {ticker_code} and quarter='q1' and analyst_name like {analyst_name} order by metric_name"
    cursor.execute(select_stmt1)
    row = cursor.fetchone()
    res_array = []
    while row:
        res_array.append(row[0])
        row = cursor.fetchone()
    return(res_array)

In [71]:
filen="C:\\trial\\2018\\may\\2 MAY\\lvs\\DPZ-Bernstein.xlsx"
print(filen)
path_list=formating(filen)    
print("The current ticker that is detected is ", path_list[3])
ticker_input = input("Would you like to proceed with this ticker? [Y]//[N]")
if(ticker_input is "Y"):
    ticker_name = path_list[3]
else:
    ticker_name = input("please enter the ticker code")
ticker_name = ticker_name.lower()
a = corpus_list
csv_file = ticker_name +"_matches.csv"
a = list(map(lambda x: x.replace("C:\\Users\\admin\\thapovan\\Thapovan2\\Corpus\\",''),a))
if(csv_file in a):
    print("The dictionary has already been built")
else:
    ticker_list.append(ticker_name.lower())
    
    ticker_dictionary_build(ticker_name)
    csv_file = ticker_name +"_matches.csv" 
ticker_dictionary = pd.read_csv(csv_file, error_bad_lines=False)
val=function1(filen,path_list)
if val==1:
    print("FAULT")
elif val==2:
    print("fault with sheet name")

C:\trial\2018\may\2 MAY\lvs\DPZ-Bernstein.xlsx
['2018', 'may', '2 MAY', 'DPZ', 'Bernstein']
The current ticker that is detected is  DPZ
Would you like to proceed with this ticker? [Y]//[N]N
please enter the ticker codelvs
The dictionary has already been built


C:\Users\admin\Anaconda3\lib\site-packages\openpyxl\reader\drawings.py:58: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)


['ModelCover', 'Annual IS', 'Guidance', 'Qtly 16-17', 'Comps', 'Units', 'Annual BS', 'Annual CF', 'Returns', 'Snapshot_DPZ', 'Disclosures']
['Annual IS', 'Annual CF', 'Annual BS']
sheet to be checked
4
6
6
excel


'C:\\trial\\2018\\may\\2 MAY\\lvs\\DPZ-Bernstein.xlsx'

Annual IS
This is excel data


...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,2017-07-19 09:30:29.750000,1999.000000,NaN,2000.000000,2001.000000,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,2009.000000,2010.000000,2011.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016A,2017E,2018E,2019E,2020E,2021E,2022E,99-'00,00-'01,01-'02,02-'03,03-'04,04-'05,05-'06,06-'07,07-'08,08-'09,09-'10,10-'11,11-'12,12-'13,13-'14,14-'15,15-'16,16-17,17-18,18-19,19-20,20-21,21-22
1,Domestic company-owned stores,378.081000,Domestic company-owned stores,377.971000,362.189000,376.533000,375.421000,382.457000,401.008000,393.406000,394.584000,357.702000,335.779000,345.636000,336.349000,323.652000,337.414000,348.497000,396.916000,439.024,490.914,535.649,582.564,628.276,679.406,737.474,-0.000290943,-0.0417545,0.0396036,-0.00295326,0.0187416,0.0485048,-0.0189572,0.00299436,-0.0934706,-0.0612884,0.0293556,-0.0268693,-0.0377495,0.042521,0.0328469,0.138937,0.106088,0.118194,0.0911256,0.0875863,0.078467,0.0813812,0.0854681
2,Domestic franchise,116.715000,Domestic franchise,120.608000,134.195000,140.667000,144.459000,155.029000,161.857000,157.740000,158.050000,153.857000,157.781000,173.344000,187.006000,195.000000,212.370000,230.192000,272.808000,312.26,348.669,381.457,412.372,440.986,472.47,508.067,0.0333548,0.112654,0.0482283,0.0269573,0.0731695,0.0440434,-0.025436,0.00196526,-0.0265296,0.0255042,0.0986367,0.0788144,0.0427473,0.0890769,0.0839196,0.185132,0.144615,0.1166,0.0940378,0.0810428,0.0693887,0.0713949,0.0753429
3,Supply Chain,603.441000,Domestic Distribution,604.096000,691.902000,676.018000,717.057000,792.026000,819.097000,762.783000,783.330000,771.107000,763.732000,960.617000,1020.904000,1039.800000,1118.873000,1262.523000,1383.161000,1544.35,1767.72,2010.83,2262.34,2467.8,2685.5,2921.05,0.00108544,0.145351,-0.022957,0.060707,0.104551,0.0341794,-0.0687513,0.0269369,-0.0156039,-0.00956417,0.257793,0.0627586,0.0185091,0.0760464,0.128388,0.0955531,0.116533,0.144641,0.137527,0.12508,0.0908135,0.0882182,0.0877136
4,International franchise,58.402000,International,63.405000,69.995000,81.762000,96.385000,116.983000,129.635000,123.390000,126.905000,142.448000,146.765000,91.300000,107.900000,120.000000,133.567000,152.621000,163.643000,176.999,211.423,248.977,286.241,330.892,379.871,434.83,0.0856649,0.103935,0.168112,0.178848,0.213705,0.108152,-0.0481737,0.0284869,0.122477,0.0303058,-0.377917,0.181818,0.112141,0.113058,0.142655,0.0722181,0.0816167,0.194488,0.177622,0.14967,0.155992,0.148019,0.144679
5,Total revenues,1156.639000,Total revenues,1166.080000,1258.281000,1274.980000,1333.322000,1446.495000,1511.597000,1437.319000,1462.869000,1425.114000,1404.057000,1570.897000,1652.159000,1678.452000,1802.224000,1993.833000,2216.528000,2472.63,2818.73,3176.91,3543.52,3867.95,4217.25,4601.43,0.00816244,0.0790692,0.0132713,0.0457591,0.0848805,0.0450067,-0.0491388,0.0177762,-0.0258089,-0.0147757,0.118827,0.0517297,0.0159143,0.0737418,0.106318,0.111692,0.115541,0.139972,0.127073,0.115398,0.0915555,0.0903054,0.0910971
6,Domestic company-owned stores,NaN,Domestic company-owned stores,289.400000,280.800000,294.951000,302.009000,313.585000,319.072000,312.129000,317.729000,298.857000,274.473000,278.296000,267.065000,247.390000,256.597000,267.385000,294.994000,331.86,381.086,418.997,459.843,497.109,538.906,586.544,NaN,-0.0297167,0.0503953,0.0239294,0.03833,0.0174976,-0.02176,0.0179413,-0.0593965,-0.0815909,0.0139285,-0.0403563,-0.0736712,0.0372165,0.0420426,0.103256,0.124972,0.148334,0.099481,0.0974844,0.0810411,0.0840808,0.0883977
7,Domestic distribution,NaN,Domestic distribution,540.300000,620.900000,604.401000,643.621000,718.937000,739.300000,681.701000,710.894000,699.669000,680.420000,778.510000,831.665000,843.328000,899.859000,1058.749000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.149176,-0.0265727,0.0648907,0.117019,0.0283238,-0.0779102,0.0428238,-0.01579,-0.0275116,0

49


,q1,q2,q3,q4,tl,Ex
2000,Nan,Nan,Nan,Nan,"[1, 27]",Nan
2001,Nan,Nan,Nan,Nan,"[1, 28]",Nan
2002,Nan,Nan,Nan,Nan,"[1, 29]",Nan
2003,Nan,Nan,Nan,Nan,"[1, 30]",Nan
2004,Nan,Nan,Nan,Nan,"[1, 31]",Nan
2005,Nan,Nan,Nan,Nan,"[1, 32]",Nan
2006,Nan,Nan,Nan,Nan,"[1, 33]",Nan
2007,Nan,Nan,Nan,Nan,"[1, 34]",Nan
2008,Nan,Nan,Nan,Nan,"[1, 35]",Nan
2009,Nan,Nan,Nan,Nan,"[1, 36]",Nan


2008


KeyboardInterrupt: 

In [65]:
len(ticker_dictionary['matches'][2][1])

1

In [ ]:
glob.glob("C:\\Users\\admin\\thapovan\\Thapovan2\\Corpus\*csv")

In [ ]:
data.head()

In [ ]:
param = str(rohan).strip("][").replace("'","").replace(", ",",")

In [145]:
a = (ticker_dictionary['matches'][2]).strip("][").replace("'","").replace(", ",",")
a


'Net Income,RevPar,Slots'

In [26]:
select_stmt1 = F"select distinct subsidiary_id from dbo.cc_actual_metrics_consolidated where " \
                    F"metric_name = 'Property, Plant & Equipment' and quarter = 'Q1' and ticker_code = 'AAP' and financial_year = 2017 and value like 1439.6200"

In [27]:
 cursor.execute(select_stmt1)

In [28]:
b=str(cursor.fetchone())
#m = []
#while b:
#    m.append(str(b.subsidiary_name))
#    b= cursor.fetchone()

In [29]:
print(b)

(0, )


In [8]:
corpus_list.replace("C:\\Users\\admin\\thapovan\\Thapovan2\\Corpus\\","")

AttributeError: 'list' object has no attribute 'replace'

In [9]:
a = corpus_list

In [10]:
 a = list(map(lambda x: x.replace("C:\\Users\\admin\\thapovan\\Thapovan2\\Corpus\\",''),a))

In [49]:
print(b)

(Decimal('0.6206'), )


In [12]:
if("lvs_matches.csv" in a):
    print(1)

1


In [46]:
string = "rohan ---> Philip"

In [47]:
string.split("--->")

['rohan ', ' Philip']

In [70]:
data.head()

AttributeError: 'list' object has no attribute 'head'

In [83]:
data.iloc[1][0]

'100% Equities Strategy '

In [76]:
data.loc[[],'Term':'word_count']

,Term,Definition,word_count
2,100% Mortgage,100% mortgage is a mortgage loan in which the ...,41


In [75]:
d = pd.DataFrame()

In [78]:
d=d.append(data.loc[[2],'Term'], ignore_index =True)

In [79]:
value = 

,Term,Definition,word_count
0,100% Mortgage,100% mortgage is a mortgage loan in which the ...,41


In [175]:
select_stmt1 = F"select metric_name from dbo.cc_actual_metrics_consolidated where " \
                    F"ticker_code = 'AAP' and (value like {valueo} or value like {value1} or value like {value3})"

In [173]:
value = -18.43
valueo= "'"+str(value*100)+"'" #orginal with 100 multiplied
value1 = "'"+str(int(value*100))+".%"+"'" # multiply by 100 and then integered
value3 = "'"+str(-1*int(value*100))+".%"+"'" # multiply by 100 and integer and negate it

In [145]:
print(value3)

'1066.%'


In [174]:
valueo = "'"+str(value)+"'" #orginal value
value1 = "'"+str(round(value,2))+"%"+"'" # round upto 2
value3 = "'"+str(-1*round(value,2))+"%"+"'" # round upto 2  and negate it

In [195]:
f= pd.DataFrame(columns=['A','B'])

In [197]:
f

,A,B


In [23]:
def a():
    a = [1,2,3]
    b= [4,5,6]
    return(a,b)


In [24]:
r,s = a()

In [25]:
r

[1, 2, 3]

In [56]:
r = "rohan[56]"

In [55]:
r - "han"

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [57]:
r1 = r[r.find("[")+1:r.find("]")]

In [52]:
int(r1)

247

In [59]:
r.replace(r1,"")

'rohan[]'

In [76]:
a = [1,2,3,4]

In [77]:
for i in a:
    i = 1

In [78]:
a

[1, 2, 3, 4]